In [5]:
!curl https://setigopublic.mybluemix.net/v1/aca/meta/all > signaldb.csv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.8M    0 18.8M    0     0  17.9M      0 --:--:--  0:00:01 --:--:-- 18.0M


In [7]:
!gunzip signaldb.csv.gz

In [9]:
!ls -alh signaldb.csv

-rw-r----- 1 sc3e-53554f95eddadf-4e28db014a7c users 122M May 15 16:36 signaldb.csv


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('signaldb.csv')
df.take(5)


[Row(UNIQUEID=u'ldn1622_1500_168_3011_19_103758', TIME=u'2013-03-02 06:03:29', ACTTYP=u'pointantswait', TGTID=u'2800031', CATALOG=u'ldn1622grid', RA2000HR=u'5.872', DEC2000DEG=u'1.087', POWER=u'401.0', SNR=None, FREQMHZ=u'1499.79070556', DRIFTHZS=u'-1.145', WIDHZ=u'2.778', POL=u'left', SIGTYP=u'CwP', PPERIODS=None, NPUL=u'0', INTTIMES=u'94.0', TSCPAZDEG=u'233.671', TSCPELDEG=u'36.754', BEAMNO=u'3', SIGCLASS=u'Cand', SIGREASON=u'PsPwrT', CANDREASON=u'SnMulBm', CONTAINER=u'setiCompAmp', OBJECTNAME=u'2013-03-02/act168/2013-03-02_06-03-45_UTC.act168.dx3011.id-19.R.archive-compamp'),
 Row(UNIQUEID=u'ldn1622_1500_168_3002_4_103698', TIME=u'2013-03-02 06:03:29', ACTTYP=u'pointantswait', TGTID=u'2800031', CATALOG=u'ldn1622grid', RA2000HR=u'5.872', DEC2000DEG=u'1.087', POWER=u'414.0', SNR=None, FREQMHZ=u'1491.86280556', DRIFTHZS=u'-0.121', WIDHZ=u'2.778', POL=u'right', SIGTYP=u'CwP', PPERIODS=None, NPUL=u'0', INTTIMES=u'94.0', TSCPAZDEG=u'233.671', TSCPELDEG=u'36.754', BEAMNO=u'3', SIGCLASS=u'C

In [2]:
df.cache()

DataFrame[UNIQUEID: string, TIME: string, ACTTYP: string, TGTID: string, CATALOG: string, RA2000HR: string, DEC2000DEG: string, POWER: string, SNR: string, FREQMHZ: string, DRIFTHZS: string, WIDHZ: string, POL: string, SIGTYP: string, PPERIODS: string, NPUL: string, INTTIMES: string, TSCPAZDEG: string, TSCPELDEG: string, BEAMNO: string, SIGCLASS: string, SIGREASON: string, CANDREASON: string, CONTAINER: string, OBJECTNAME: string]

In [3]:
df.createOrReplaceTempView("signaldb")

In [4]:
df2 = spark.sql("select container, objectname from signaldb where RA2000hr == 0.03 and dec2000deg == 66.306")

In [8]:
raw_loc = df2.collect()

In [9]:
raw_loc

[Row(container=u'setiCompAmp', objectname=u'2013-03-14/act12/2013-03-14_20-43-56_UTC.act12.dx1012.id-1.R.archive-compamp'),
 Row(container=u'setiCompAmp', objectname=u'2013-03-14/act12/2013-03-14_20-43-56_UTC.act12.dx1012.id-1.L.archive-compamp'),
 Row(container=u'setiCompAmp', objectname=u'2013-03-14/act10/2013-03-14_20-37-33_UTC.act10.dx1000.id-1.L.archive-compamp'),
 Row(container=u'setiCompAmp', objectname=u'2013-03-14/act10/2013-03-14_20-37-33_UTC.act10.dx1012.id-1.R.archive-compamp'),
 Row(container=u'setiCompAmp', objectname=u'2013-03-14/act10/2013-03-14_20-37-32_UTC.act10.dx1000.id-0.R.archive-compamp'),
 Row(container=u'setiCompAmp', objectname=u'2013-03-14/act15/2013-03-14_20-52-29_UTC.act15.dx1006.id-1.L.archive-compamp'),
 Row(container=u'setiCompAmp', objectname=u'2013-03-14/act10/2013-03-14_20-37-33_UTC.act10.dx1017.id-2.R.archive-compamp'),
 Row(container=u'setiCompAmp', objectname=u'2013-03-14/act10/2013-03-14_20-37-32_UTC.act10.dx1006.id-1.L.archive-compamp'),
 Row(con